In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

%cd /content/drive/MyDrive/Colab\ Notebooks/MovieSentimentalAnalysis/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MovieSentimentalAnalysis


In [2]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 448 kB 49.2 MB/s 


In [3]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-10zjx0ng
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-10zjx0ng
     |████████████████████████████████| 460.3 MB 7.7 kB/s 
     |████████████████████████████████| 4.0 MB 47.4 MB/s 
     |████████████████████████████████| 132 kB 44.2 MB/s 
     |████████████████████████████████| 462 kB 62.8 MB/s 
     |████████████████████████████████| 1.2 MB 72.8 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268580 sha256=55b4502c2a2deca390c315f21ce3831408af15c5a6d7a93626c30715ef5aa49f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lunthd6y/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68723 sha256=ee35e43c7eb7482df85a9e5af539d3151c74876016ac9734f60e241c9082f906
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d

In [4]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
#nltk.download('all') # 처음 실행 시 주석을 제거하고 실행해주세요!
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Okt 
from pykospacing import Spacing

In [5]:
data = pd.read_csv('dataset/train.csv') # train 데이터 불러오기
data.head() # 데이터 확인하기

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [6]:
data.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

#1. 전처리 (preprocess)

* 특수문자 제거



In [7]:
#학습 셋 전처리
data['document'] = data['document'].str.lower()
data['preprocessed'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-z ]"," ") # 특수문자를 공백으로 치환
data.sample(5)

,id,document,label,preprocessed
4786,4787,무엇을 위한 영화인가? 헛웃음만이 나올뿐이다.,0,무엇을 위한 영화인가 헛웃음만이 나올뿐이다
1399,1400,아직도 그립다...폴워커...,1,아직도 그립다 폴워커
3981,3982,외국영화보면서 운적은 처음이네요ㅜㅜ,1,외국영화보면서 운적은 처음이네요ㅜㅜ
736,737,졸잼 개 잼있다 꿀잼 안본사람들 꼭보셈,1,졸잼 개 잼있다 꿀잼 안본사람들 꼭보셈
677,678,지훈 오빠 무 멋있어요/``!!,1,지훈 오빠 무 멋있어요


* 띄어쓰기 (Using pykospacing)

In [8]:
# 인스턴스 할당
spacing = Spacing()

spaced = []
for sentence in data['preprocessed']:
    sentence = spacing(sentence)
    spaced.append(sentence)
data['spaced'] = pd.DataFrame(spaced)

data.sample(5)

,id,document,label,preprocessed,spaced
2415,2416,0점 주고싶네요.... 쓰레기 ㅠㅠ 내 시간을 낭비했어,0,0점 주고싶네요 쓰레기 ㅠㅠ 내 시간을 낭비했어,0점 주고 싶네요 쓰레기 ㅠㅠ 내 시간을 낭비했어
4860,4861,로맨틱 코미디 영화의 최고라고 말하고 싶다,1,로맨틱 코미디 영화의 최고라고 말하고 싶다,로맨틱 코미디 영화의 최고라고 말하고 싶다
3127,3128,이런영화가 나와주기만하는거만으로도 고맙다,1,이런영화가 나와주기만하는거만으로도 고맙다,이런 영화가 나와주기만 하는 거만으로도 고맙다
1763,1764,배드신이 너무 많아 어린아이가 보기에 그러함,0,배드신이 너무 많아 어린아이가 보기에 그러함,배드신이 너무 많아 어린 아이가 보기에 그러함
4249,4250,와 방금 광고났는데 채널cgv가 오늘밤 10시에 해준데요.,1,와 방금 광고났는데 채널cgv가 오늘밤 10시에 해준데요,와 방금 광고났는데 채널 cgv가 오늘 밤 10시에 해준데요


* 형태소 분석 (using Okt)

In [9]:
# 인스턴스 할당
okt = Okt()

tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in data['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
data["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.


In [10]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in data['preprocessed']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] not in ("Josa", "Foreign", "Suffix")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
data["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.


In [11]:
data.sample(5)

,id,document,label,preprocessed,spaced,tokenized_stem,main_pos
3082,3083,오랜만에 스릴러다 더 재미 있었네 ㅋㅋ,1,오랜만에 스릴러다 더 재미 있었네 ㅋㅋ,오랜만에 스릴러다 더 재미 있었네 ㅋㅋ,오랜 만 에 스릴러 다 더 재미 있다 ㅋㅋ,오랜 만 스릴러 더 재미 있었네 ㅋㅋ
3177,3178,평점이 넘 낮다 생각했던것 보다 훨씬 재미있었음 잘근잘근 씹히는 맛,1,평점이 넘 낮다 생각했던것 보다 훨씬 재미있었음 잘근잘근 씹히는 맛,평점이 넘 낮다 생각했던 것 보다 훨씬 재미있었음 잘근잘근 씹히는 맛,평점 이 넘다 낮다 생각 하다 보다 훨씬 재미있다 잘근잘근 씹히다 맛,평점 넘 낮다 생각 했던것 보다 훨씬 재미있었음 잘근잘근 씹히는 맛
498,499,여자들 옷벗은 거나 계속 찍는게 이것 보다는 났겠다.,0,여자들 옷벗은 거나 계속 찍는게 이것 보다는 났겠다,여자들 옷 벗은 거나 계속 찍는 게 이것 보다는 났겠다,여자 들 옷벗다 거나 계속 찍다 이 것 보다는 나다,여자 옷벗은 거나 계속 찍는게 이 것 났겠다
457,458,영화를 보다 십분만에 중단하기 참으로 어려운 일인데 이 영화는 해냈다,0,영화를 보다 십분만에 중단하기 참으로 어려운 일인데 이 영화는 해냈다,영화를 보다 십분 만에 중단하기 참으로 어려운 일인데 이 영화는 해냈다,영화 를 보다 십분 만에 중단 하다 참으로 어렵다 일 인데 이 영화 는 해내다,영화 보다 십분 중단 하기 참으로 어려운 일 이 영화 해냈다
1353,1354,재미없다 이런 드라마에 십점이라니.,0,재미없다 이런 드라마에 십점이라니,재미없다 이런 드라마에 십 점이라니,재미없다 이렇다 드라마 에 십 점 이라니,재미없다 이런 드라마 십 점


* Okt - Example

In [12]:
txt = '10년간 지금까지의 영화들이 그랬듯 이 영화 역시 일본에 대한 미화는 여전하다'
pos = okt.pos(txt)
print(pos)

[('10년', 'Number'), ('간', 'Foreign'), ('지금', 'Noun'), ('까지의', 'Josa'), ('영화', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('그랬듯', 'Adjective'), ('이', 'Noun'), ('영화', 'Noun'), ('역시', 'Noun'), ('일본', 'Noun'), ('에', 'Josa'), ('대한', 'Noun'), ('미화', 'Noun'), ('는', 'Josa'), ('여전하다', 'Adjective')]


* Split Dataset

In [14]:
#preprocessed, spaced, tokenized_stem, main_pos 중에 학습할 데이터셋을 선택
X = data['tokenized_stem']
y = np.array(data.label)

train, val, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

# 2. Train & Validation - Logistic Regression

In [15]:
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)
predicted = lgs.predict(X_eval)
print(f"Accuracy: {lgs.score(X_eval, y_eval)}" )

Accuracy: 0.88


In [16]:
df = pd.DataFrame(val)
#df.insert(1, 'LR', predicted)
df.reset_index(inplace = True)
df.loc[:,'answer'] = y_eval
df.loc[:,'LR'] = predicted

In [19]:
df.sample(5)

,index,tokenized_stem,answer,LR
436,3429,멋있다 영화 이다 부족하다 뭐 든지 하다 수 있다 교훈,1,1
583,1510,감동 이에요 정말로 재미있다,1,1
402,2629,남 고딩 들 이 이렇게 사랑 스럽게 나오다 영화 라니,1,0
787,432,시원하다 아주 미미하다 엉 성하다 스토리 의 영화,0,0
703,1500,재밌다 본 걸 로 기억 에 남다 영화,1,1


# 3. Train & Validation - Random Forest

In [20]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
forest = RandomForestClassifier(n_estimators = 100, random_state = 2022)
forest = forest.fit(X_train, y_train)

predicted = forest.predict(X_eval)
print(f"Accuracy: {forest.score(X_eval, y_eval)}" )

Accuracy: 0.829


In [21]:
df.loc[:,'RF'] = predicted
df.sample(5)

,index,tokenized_stem,answer,LR,RF
953,1637,미스 캐스팅 이 결정 적다 영화 자체 도 별 흥미 없다,0,0,0
52,4467,왜 상 받다 진짜 지루하다 끝 까지 보다 욕 나오다 보다 암,0,0,0
380,4391,묻다 따다 않다 10 점 만점 이다,1,1,1
882,2167,별 점 에 속 지 말 자 0 점 옆 커플 싸우다 혼자 보다 존 니 다행,0,0,0
144,4140,이 거 안보 면 초 후회 하다 초강추 양 채 니 미모 는 와우 ㅋㅋ,1,1,0


# 4. Train & Validation - Support Vector Machine

In [22]:
#Support Vector Machine
from sklearn import svm

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
clf = svm.SVC()
clf = clf.fit(X_train, y_train)

predicted = clf.predict(X_eval)
print(f"Accuracy: {clf.score(X_eval, y_eval)}" )

Accuracy: 0.894


In [23]:
df.loc[:,'SVM'] = predicted
df.sample(5)

,index,tokenized_stem,answer,LR,RF,SVM
477,2880,꼭 챙기다 보다 여운 도 남고 예술 과 하다 좋다 시간 이다,1,1,1,1
447,2479,진짜 비디오 로 보다 그나마 다행 이다,0,0,1,0
136,1576,정말 최고 의 영화 이다 사람 을 깜짝 놀라다 하다 ㅎㅎ 최고 이다,1,1,1,1
466,842,별 의미 없다 섹스 능력 이 좋다 소문나다 서로 하다 경쟁 하다 데요,0,1,0,1
442,1947,감독 태원 이 사람 제 정신 이냐 이 게 영화 냐,0,0,0,0


# 5. Train & Validation - Naive Bayes

In [24]:
from sklearn.naive_bayes import BernoulliNB

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
NB = BernoulliNB()
NB = NB.fit(X_train, y_train)

predicted = NB.predict(X_eval)
print(f"Accuracy: {NB.score(X_eval, y_eval)}" )

Accuracy: 0.889


In [25]:
df.loc[:,'NB'] = predicted
df.sample(5)

,index,tokenized_stem,answer,LR,RF,SVM,NB
2,1918,리암 니 슨 연기 몰입도 교훈,1,1,1,1,1
665,2592,한국영 화중 에 제일 무섭다 영화 는 요 라고 묻다 서슴없이 여 곡성,1,1,1,1,1
126,2976,ㅠㅠ 수트 어쩔 거 에요 ㅠ 정말 재밌다 ㅎㅎ,1,1,1,1,1
185,3857,초딩 때 보다 같다 몇 년 이 지난 지금 도 기억 에 남다 영화,1,1,1,1,1
723,4278,원작 보다 재밋다 는 구라 좀 치다,0,0,1,0,0


In [ ]:
out = val[val['label']!=val['answer']]
out.to_csv("wrong.csv", index = False)